In [1]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [2]:
def get_warped_images(image, pts):
    rect = order_points(pts)
    tl, tr, br, bl = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(widthA, widthB)

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(heightA, heightB)

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
    [0, maxHeight - 1]], dtype = "float32")
    
    rect = np.array(rect, dtype = "float32")
    M = cv2.getPerspectiveTransform(src=rect, dst=dst)
    warped = cv2.warpPerspective(image, M=M, dsize=(int(maxWidth), int(maxHeight)))

    return warped

In [3]:
# image_patterns = ['*.jpg', '*.png', '*.jpeg', '*.JPG', '*.PNG', '*.JPEG']
patterns = ['*.json']
input_dir = Path('/home/trucly/Documents/DATASET/handwritten/merge_bg/vnondb_vtp/create_background')
output_dir = Path('/home/trucly/Documents/DATASET/handwritten/merge_bg/vnondb_vtp/textline')
paths = []
for pattern in patterns:
    paths += list(input_dir.glob(f'**/{pattern}'))      

In [4]:
cnt = 0
for path in paths:
    filename = str(path)
    with open(filename, 'r') as f:
        obj = json.load(f)
        
    image_name = obj['imagePath']
    image = cv2.imread(str(input_dir.joinpath(image_name)))
    shapes = obj['shapes']

    for x in shapes:
        cnt += 1
        point = x['points']

#         value = x['value']
        if len(point) == 4:
            line = get_warped_images(image, order_points(point))
        elif len(point) == 2:
            line = image[int(point[0][1]):int(point[1][1]), int(point[0][0]):int(point[1][0])]
        else:
            continue
            
#         image_path = output_dir.joinpath(path.stem + '_' + str(cnt) + '.jpg')
        image_path = output_dir.joinpath('bg_' + str(cnt) + '.jpg')
        cv2.imwrite(str(image_path), line)
#             f.write(value)

#         label_path = output_dir.joinpath('label.txt')
#         with open(str(label_path), 'a') as f:
#             f.write(path.stem + '_' + str(cnt) + '.jpg' + '\t' + value + '\n')